In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/cmlscratch/sdooley1/merge_timm/FR-NAS/Checkpoints/')
from analysis import *
import glob
import string
import plotly.graph_objects as go

In [2]:
import pickle

with open('smac_032_test.pkl','rb') as f:
    smac_032_test = pickle.load(f)

with open('smac_032_val.pkl','rb') as f:
    smac_032_val = pickle.load(f)
    
with open('smac_728_test.pkl','rb') as f:
    smac_728_test = pickle.load(f)

with open('smac_728_val.pkl','rb') as f:
    smac_728_val = pickle.load(f)

In [3]:
test_metadata = pd.read_csv('../Checkpoints/vggface2_test_identities_gender.csv')
epochs = ['epoch_'+str(i) for i in [0]]


pd_dict_list = []

for df,name in [(smac_032_test, 'MO-ASHA_032_test'),
                (smac_728_test, 'NSGA-II_728_test')]:

    df = df[[x for x in df.columns if 'nearest_by_id' in x]]
    df.columns = ['epoch_'+x.split('_')[1] for x in df.columns]
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'ids'})
    df = df.rename(columns = {'epoch_by':'epoch_0'})
    pd_dict_list += [{'experiment':name,
                      'df': df}]

acc_df, acc_disp_df, rank_df = analyze_files_pd(pd_dict_list, test_metadata, epochs=epochs)
_, acc_disp_ratio_df, rank_ratio_df = analyze_files_pd(pd_dict_list, test_metadata, ratio=True, epochs=epochs)
err_df, error_ratio_df, _ = analyze_files_pd(pd_dict_list, test_metadata, ratio=True, error=True, epochs=epochs)

df_test = acc_df
df_test.columns = ['Accuracy']
for df, name in [(rank_df, 'Rank Disparity'), 
                 (acc_disp_df, 'Disparity'),
                 (acc_disp_ratio_df, 'Ratio'),
                 (rank_ratio_df, 'Rank Ratio'),
                 (error_ratio_df, 'Error Ratio')]:
    cols = df_test.columns
    df_test = pd.concat([df_test,df], axis=1)
    df_test.columns = list(cols)+[name]

In [4]:
val_metadata = pd.read_csv('../Checkpoints/vggface2_val_identities_gender.csv')
epochs = ['epoch_'+str(i) for i in [0]]


pd_dict_list = []

for df,name in [(smac_032_val, 'MO-ASHA_032_val'),
                (smac_728_val, 'NSGA-II_728_val')]:

    df = df[[x for x in df.columns if 'nearest_by_id' in x]]
    df.columns = ['epoch_'+x.split('_')[1] for x in df.columns]
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'ids'})
    df = df.rename(columns = {'epoch_by':'epoch_0'})
    pd_dict_list += [{'experiment':name,
                      'df': df}]

acc_df, acc_disp_df, rank_df = analyze_files_pd(pd_dict_list, test_metadata, epochs=epochs)
_, acc_disp_ratio_df, rank_ratio_df = analyze_files_pd(pd_dict_list, test_metadata, ratio=True, epochs=epochs)
err_df, error_ratio_df, _ = analyze_files_pd(pd_dict_list, test_metadata, ratio=True, error=True, epochs=epochs)

df_val = acc_df
df_val.columns = ['Accuracy']
for df, name in [(rank_df, 'Rank Disparity'), 
                 (acc_disp_df, 'Disparity'),
                 (acc_disp_ratio_df, 'Ratio'),
                 (rank_ratio_df, 'Rank Ratio'),
                 (error_ratio_df, 'Error Ratio')]:
    cols = df_val.columns
    df_val = pd.concat([df_val,df], axis=1)
    df_val.columns = list(cols)+[name]

In [5]:
random_ngsa2 = pd.concat([df_val,df_test])
print(random_ngsa2)

                  Accuracy  Rank Disparity  Disparity     Ratio  Rank Ratio  \
MO-ASHA_032_val   0.952843        0.005882   0.026601  0.028313    0.012479   
NSGA-II_728_val   0.895261        0.111569   0.032222  0.036652    0.175871   
MO-ASHA_032_test  0.934739        0.390588   0.048562  0.053338    0.448144   
NSGA-II_728_test  0.868105        0.599085   0.085752  0.103913    0.490213   

                  Error Ratio  
MO-ASHA_032_val      0.440000  
NSGA-II_728_val      0.266631  
MO-ASHA_032_test     0.542336  
NSGA-II_728_test     0.490651  


In [6]:
b = []
for set in ['test', 'val']: #disparity, errorratio, rankdisparity, rankratio, ratio
    a = {'index': f'SMAC_301_{set}'}
    for short_metric, metric in [('disparity', 'Disparity'),
                                 ('ratio', 'Ratio'),
                                 ('rankratio', 'Rank Ratio'),
                                 ('errorratio', 'Error Ratio'),
                                 ('rankdisparity', 'Rank Disparity')]:

        df = pd.read_csv(f'../Checkpoints/vggface_phase2pareto_{short_metric}_{set}.csv')
        row = df[df['index'] == 'Checkpoints_Edges_301_LR_0.13828312564892567_Head_CosFace_Optimizer_SGD']
        a[metric] = row['Disparity_mean'].values[0]
        a['Accuracy'] = 1-row['Accuracy_mean'].values[0]
    b += [a]
b = pd.DataFrame(b).set_index('index')
b

,Disparity,Accuracy,Ratio,Rank Ratio,Error Ratio,Rank Disparity
index,,,,,,
SMAC_301_test,0.030087,0.963366,0.031727,0.367554,0.582215,0.230327
SMAC_301_val,0.009412,0.974989,0.009700,0.027789,0.317078,0.009717


In [7]:
random_ngsa2 = pd.concat([random_ngsa2,b])
random_ngsa2.to_csv('random_ngsa2_results.csv')

In [8]:
print(random_ngsa2[random_ngsa2.index.str.contains("test")].to_markdown())

|                  |   Accuracy |   Rank Disparity |   Disparity |     Ratio |   Rank Ratio |   Error Ratio |
|:-----------------|-----------:|-----------------:|------------:|----------:|-------------:|--------------:|
| MO-ASHA_032_test |   0.934739 |         0.390588 |   0.0485621 | 0.0533381 |     0.448144 |      0.542336 |
| NSGA-II_728_test |   0.868105 |         0.599085 |   0.0857516 | 0.103913  |     0.490213 |      0.490651 |
| SMAC_301_test    |   0.963366 |         0.230327 |   0.0300871 | 0.0317269 |     0.367554 |      0.582215 |
